<a href="https://colab.research.google.com/github/WoojinJeonkr/DeepLearning/blob/main/Sequence_to_sequence_learning_for_performing_number_addition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 숫자 덧셈을 위한 Sequence to sequence learning
- 내용 출처: https://keras.io/examples/nlp/addition_rnn/
- 목표: Input: "535+61"이라면 Output: "596"으로 출력

## 01. 설정

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
# 파라미터 정의
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

In [3]:
# 최대 길이 정의
MAXLEN = DIGITS + 1 + DIGITS

## 02. 데이터 생성

In [4]:
class CharacterTable:
  # Initialize character table
  def __init__(self, chars):
    self.chars = sorted(set(chars))
    self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
    self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

  # 원핫 인코딩
  def encode(self, C, num_rows):
      x = np.zeros((num_rows, len(self.chars)))
      for i, c in enumerate(C):
          x[i, self.char_indices[c]] = 1
      return x

  # vector 또는 2D 배열을 문자 출력으로 디코딩
  def decode(self, x, calc_argmax=True):
    if calc_argmax:
        x = x.argmax(axis=-1)
    return "".join(self.indices_char[x] for x in x)

In [6]:
# 패딩
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()

    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)

    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)

    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))

Generating data...
Total questions: 50000


## 03. 데이터 벡터화

In [7]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

In [9]:
# 훈련 데이터와 검증 데이터 분할
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


## 모델 구축

In [10]:
print("Build model...")
num_layers = 1

model = keras.Sequential()
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(num_layers):
  model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 repeat_vector (RepeatVector  (None, 4, 128)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 4, 128)            131584    
                                                                 
 dense (Dense)               (None, 4, 12)             1548      
                                                                 
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


## 모델 훈련

In [11]:
epochs = 30
batch_size = 32

In [13]:
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )

    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 1
1407/1407 [==============================] - 26s 18ms/step - loss: 1.6999 - accuracy: 0.3728 - val_loss: 1.4963 - val_accuracy: 0.4451
Q 71+19   T 90   ☒ 11  
Q 76+17   T 93   ☒ 11  
Q 247+32  T 279  ☒ 388 
Q 565+53  T 618  ☒ 668 
Q 498+21  T 519  ☒ 598 
Q 133+964 T 1097 ☒ 1044
Q 994+828 T 1822 ☒ 1805
Q 55+943  T 998  ☒ 104 
Q 50+963  T 1013 ☒ 100 
Q 69+76   T 145  ☒ 112 

Iteration 2
1407/1407 [==============================] - 23s 17ms/step - loss: 1.3011 - accuracy: 0.5132 - val_loss: 1.1322 - val_accuracy: 0.5720
Q 98+342  T 440  ☒ 419 
Q 4+960   T 964  ☒ 960 
Q 870+72  T 942  ☒ 939 
Q 980+347 T 1327 ☒ 1301
Q 842+78  T 920  ☒ 939 
Q 0+67    T 67   ☒ 77  
Q 434+5   T 439  ☒ 449 
Q 61+827  T 888  ☒ 899 
Q 591+439 T 1030 ☒ 1011
Q 383+65  T 448  ☒ 459 

Iteration 3
1407/1407 [==============================] - 23s 17ms/step - loss: 1.0269 - accuracy: 0.6184 - val_loss: 0.9460 - val_accuracy: 0.6483
Q 4+191   T 195  ☒ 191 
Q 48+767  T 815  ☒ 811 
Q 511+57  T 568  ☒ 561 
Q 30